# Worksheet 12

Name:  Jasper Hoong
UID: U91969628

### Topics

- Introduction to Classification
- K Nearest Neighbors

### Introduction to Classification

a) For the following examples, say whether they are or aren't an example of classification.

1. Predicting whether a student will be offered a job after graduating given their GPA.
2. Predicting how long it will take (in number of months) for a student to be offered a job after graduating, given their GPA.
3. Predicting the number of stars (1-5) a person will assign in their yelp review given the description they wrote in the review.
4. Predicting the number of births occuring in a specified minute.

1. Yes
2. No
3. Yes
4. No

b) Given a dataset, how would you set things up such that you can both learn a model and get an idea of how this model might perform on data it has never seen?

We can use the model to try and predict some of the data in the dataset to train the model. Separate the dataset into two sets of data, with one for setting up the model and the other for training it and testing it. 

c) In your own words, briefly explain:

- underfitting
- overfitting

and what signs to look out for for each.

Underfitting is where the model makes too many mistakes and is too simple, overfitting is the opposite where it is very complex but makes little to no mistakes. A sign of underfitting if there are a lot of outliers and the opposite for overfitting.\

### K Nearest Neighbors

In [14]:
import numpy as np
import matplotlib.pyplot as plt

data = {
    "Attribute A" : [3.5, 0, 1, 2.5, 2, 1.5, 2, 3.5, 1, 3, 2, 2, 2.5, 0.5, 0., 10],
    "Attribute B" : [4, 1.5, 2, 1, 3.5, 2.5, 1, 0, 3, 1.5, 4, 2, 2.5, 0.5, 2.5, 10],
    "Class" : [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
}

a) Plot the data in a 2D plot coloring each scatter point one of two colors depending on its corresponding class.

In [ ]:
colors = np.array([x for x in 'bgrcmyk'])
plt.scatter(data["Attribute A"], data["Attribute B"], color=colors[data["Class"]].tolist())
plt.show()

Outliers are points that lie far from the rest of the data. They are not necessarily invalid points however. Imagine sampling from a Normal Distribution with mean 10 and variance 1. You would expect most points you sample to be in the range [7, 13] but it's entirely possible to see 20 which, on average, should be very far from the rest of the points in the sample (unless we're VERY (un)lucky). These outliers can inhibit our ability to learn general patterns in the data since they are not representative of likely outcomes. They can still be useful in of themselves and can be analyzed in great depth depending on the problem at hand.

b) Are there any points in the dataset that could be outliers? If so, please remove them from the dataset.

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for Attribute A and Attribute B
Q1_A = np.percentile(data["Attribute A"], 25)
Q3_A = np.percentile(data["Attribute A"], 75)
IQR_A = Q3_A - Q1_A

Q1_B = np.percentile(data["Attribute B"], 25)
Q3_B = np.percentile(data["Attribute B"], 75)
IQR_B = Q3_B - Q1_B

# Define lower and upper bounds for potential outliers
lower_bound_A = Q1_A - 1.5 * IQR_A
upper_bound_A = Q3_A + 1.5 * IQR_A

lower_bound_B = Q1_B - 1.5 * IQR_B
upper_bound_B = Q3_B + 1.5 * IQR_B

# Identify potential outliers
outliers_A = [index for index, value in enumerate(data["Attribute A"]) if value < lower_bound_A or value > upper_bound_A]
outliers_B = [index for index, value in enumerate(data["Attribute B"]) if value < lower_bound_B or value > upper_bound_B]

# Remove outliers from the dataset
filtered_data = {
    "Attribute A": [data["Attribute A"][i] for i in range(len(data["Attribute A"])) if i not in outliers_A],
    "Attribute B": [data["Attribute B"][i] for i in range(len(data["Attribute B"])) if i not in outliers_B],
    "Class": [data["Class"][i] for i in range(len(data["Class"])) if i not in outliers_A and i not in outliers_B]
}

print("Filtered Data:")
print(filtered_data)

Noise points are points that could be considered invalid under the general trend in the data. These could be the result of actual errors in the data or randomness that we could attribute to oversimplification (for example if missing some information / feature about each point). Considering noise points in our model can often lead to overfitting.

c) Are there any points in the dataset that could be noise points?

Yes, there are

For the following point

|  A  |  B  |
|-----|-----|
| 0.5 |  1  |

d) Plot it in a different color along with the rest of the points in the dataset.

In [ ]:
import matplotlib.pyplot as plt

# Original filtered data (excluding outliers)
filtered_data = {
    "Attribute A": [3.5, 1, 2.5, 2, 1.5, 2, 3.5, 1, 2, 2, 2.5, 0.5, 10],
    "Attribute B": [4, 2, 1, 3.5, 2.5, 1, 0, 3, 1.5, 2, 2.5, 2.5, 10],
    "Class": [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
}

# New point to be plotted
new_point = [0.5, 1]

# Scatter plot with different colors for each class
plt.scatter(filtered_data["Attribute A"], filtered_data["Attribute B"], c=filtered_data["Class"], cmap='viridis', label="Data Points")
plt.scatter(new_point[0], new_point[1], color='red', label="New Point (0.5, 1)")
plt.xlabel("Attribute A")
plt.ylabel("Attribute B")
plt.title("Scatter Plot of Attribute A vs Attribute B")
plt.colorbar(label="Class")
plt.legend()
plt.show()

e) Write a function to compute the Euclidean distance from it to all points in the dataset and pick the 3 closest points to it. In a scatter plot, draw a circle centered around the point with radius the distance of the farthest of the three points.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Original filtered data (excluding outliers)
filtered_data = {
    "Attribute A": [3.5, 1, 2.5, 2, 1.5, 2, 3.5, 1, 2, 2, 2.5, 0.5, 10],
    "Attribute B": [4, 2, 1, 3.5, 2.5, 1, 0, 3, 1.5, 2, 2.5, 2.5, 10],
    "Class": [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
}

def n_closest_to(example, n):
    distances = []
    for point in zip(filtered_data["Attribute A"], filtered_data["Attribute B"]):
        distance = np.linalg.norm(np.array(example) - np.array(point))
        distances.append(distance)
    closest_indices = np.argsort(distances)[:n]
    closest_points = [(filtered_data["Attribute A"][i], filtered_data["Attribute B"][i]) for i in closest_indices]
    return closest_points

# New point
location = (0.5, 1)

# Compute the 3 closest points to the new point
closest_points = n_closest_to(location, 3)

# Compute the radius (distance to the farthest of the 3 closest points)
radius = max([np.linalg.norm(np.array(location) - np.array(point)) for point in closest_points])

# Plotting
_, axes = plt.subplots()
axes.scatter(filtered_data["Attribute A"], filtered_data["Attribute B"], c=filtered_data["Class"], cmap='viridis', label="Data Points")
axes.scatter(location[0], location[1], color='red', label="New Point (0.5, 1)")
cir = Circle(location, radius, fill=False, alpha=0.8)
axes.add_patch(cir)
axes.set_aspect('equal') # necessary so that the circle is not oval
plt.xlabel("Attribute A")
plt.ylabel("Attribute B")
plt.title("Scatter Plot with Circle")
plt.colorbar(label="Class")
plt.legend()
plt.show()

f) Write a function that takes the three points returned by your function in e) and returns the class that the majority of points have (break ties with a deterministic default class of your choosing). Print the class assigned to this new point by your function.

In [ ]:
def majority(points):
    # Count the occurrences of each class in the given points
    class_counts = {}
    for point in points:
        _, _, class_label = point
        class_counts[class_label] = class_counts.get(class_label, 0) + 1
    
    # Determine the majority class (break ties with a default class of 1)
    majority_class = max(class_counts, key=class_counts.get, default=1)
    return majority_class


g) Re-using the functions from e) and f), you should be able to assign a class to any new point. In this exercise we will implement Leave-one-out cross validiation in order to evaluate the performance of our model.

For each point in the dataset:

- consider that point as your test set and the rest of the data as your training set
- classify that point using the training set
- keep track of whether you were correct with the use of a counter

Once you've iterated through the entire dataset, divide the counter by the number of points in the dataset to report an overall testing accuracy.

In [ ]:
count = 0

# Iterate through the entire dataset
for i in range(len(filtered_data["Attribute A"])):
    # Consider the i-th point as the test set
    test_point = (filtered_data["Attribute A"][i], filtered_data["Attribute B"][i], filtered_data["Class"][i])

    # Create the training set by excluding the test point
    training_data = [(filtered_data["Attribute A"][j], filtered_data["Attribute B"][j], filtered_data["Class"][j])
                     for j in range(len(filtered_data["Attribute A"])) if j != i]

    # Get the 3 closest points using the training set
    closest_points = n_closest_to(test_point[:2], 3)

    # Predict the class of the test point using the majority function
    predicted_class = majority(closest_points)

    # Compare the predicted class with the actual class
    if predicted_class == test_point[2]:
        count += 1

# Calculate the overall accuracy
overall_accuracy = count / len(filtered_data["Attribute A"]) * 100
print("Overall accuracy =", overall_accuracy, "%")